# Elaborate statistics features for Silvereye

This is the first of a series of notebooks to tease the statistics on AWRA and related data for Silvereye.

The current notebook is for the initial trial-error.

## Purpose

Merge capabilities from:

* Ben's code in the prototype app
* Ramneek's stat on daily data.
* possibly some of the princeton-chelsa blend code (optional)

Plan is:

* Subset AWRA daily data sets to a more manageable size for initial elaboration. ACT or Tassie.
* Elaborate on use cases:
    * Existing features infered from BoM web served maps
    * compare periods (e.g. recent past ) to similar historical periods* Subset AWRA daily data sets to a more manageable size for initial elaboration. ACT or Tassie.

## Dependencies imports


In [ ]:
import xarray as xr
import os
import sys
import pandas as pd
from functools import wraps
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns  # noqa, pandas aware plotting library

In [ ]:
if ('SP_SRC' in os.environ):
    root_src_dir = os.environ['SP_SRC']
elif sys.platform == 'win32':
    root_src_dir = r'C:\src\csiro\stash\silverpieces'
else:
    root_src_dir = '/home/per202/src/csiro/stash/silverpieces'

pkg_src_dir = root_src_dir
sys.path.append(pkg_src_dir)

In [ ]:
if ('SP_DATA' in os.environ):
    root_data_dir = os.environ['SP_DATA']
elif sys.platform == 'win32':
    root_data_dir = r'C:\data\silverpieces'
else:
    root_data_dir = '/home/per202/data/silverpieces'


In [ ]:
#from silverpieces.blah import *

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:

from siphon.catalog import TDSCatalog
import requests
# from distributed import Client, LocalCluster
# from flask import Flask, redirect, url_for, request, render_template, jsonify, abort
# from flask_cors import CORS
# import intake
# import regionmask
# import random
# from flask import jsonify, make_response
# import json
# import uuid
# from flask.logging import default_handler
# import geopandas as gpd
# from owslib.wps import WebProcessingService
# from owslib.wps import printInputOutput
# from birdy import WPSClient
# import birdy
# import tarfile

# import tempfile
# import shutil
# import urllib
import urllib.request

# from urllib.parse import urlparse
# from datetime import datetime
# from dateutil.relativedelta import relativedelta


I tried to use [xarray-tips-and-tricks](https://rabernat.github.io/research_computing_2018/xarray-tips-and-tricks.html) by similarity but while this looked OK, retrieving values ended up in a `RuntimeError: NetCDF: Access failure`. Park this. This may be an issue with the thredds server serving awra data.

In [ ]:
# #base_url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep.reanalysis/surface/air.sig995'
# base_url = 'http://data-mel.it.csiro.au/thredds/dodsC/catch_all/lw_oznome/AWRA-L_historical_data/qtot/qtot_avg_'
# f = ['http://data-mel.it.csiro.au/thredds/dodsC/catch_all/lw_oznome/AWRA-L_historical_data/qtot/qtot_avg_' + str(yr) + '.nc' for yr in [2011, 2012]]
# files = [f'{base_url}{year}.nc' for year in range(2010, 2015)]
# files

In [ ]:
# ds = xr.open_mfdataset(files)
# ds

In [ ]:
# x = ds.qtot_avg
# x[1,1,1].values

In [ ]:
cat_url = 'http://data-mel.it.csiro.au/thredds/catalog/catch_all/lw_oznome/AWRA-L_historical_data/qtot/catalog.xml'

In [ ]:
cat = TDSCatalog(cat_url)

In [ ]:
ds_names = [str(x) for x in cat.datasets]
m_indices = [i for i in range(len(ds_names)) if ds_names[i].startswith('qtot') and ds_names[i].endswith('nc')]
# somehow sorted
m_indices.reverse()

In [ ]:
ds = [cat.datasets[i] for i in m_indices]

In [ ]:
ds[0].access_urls

In [ ]:
dsets = [cds.remote_access(use_xarray=True).reset_coords(drop=True).chunk({'latitude': 681, 'longitude': 841})  # 731, latitude: 681, longitude: 841
         for cds in ds[:3]] # eventually want to use the whole catalog here
#dsets = [cds.remote_access(use_xarray=True).reset_coords(drop=True)
#         for cds in ds[:3]] # eventually want to use the whole catalog here

In [ ]:
len(dsets)

In [ ]:
blah = xr.auto_combine(dsets)
blah

In [ ]:
x = blah.qtot_avg

In [ ]:
x['time'].values

`RuntimeError: NetCDF: Access failure`. That said, it worked one time out of ~10 so this is probably more a problem with the thredds server.

In [ ]:
# Random but frequently ends up RuntimeError: NetCDF: Access failure
m = x[1,:,:].values

In [ ]:
plt.imshow(m)

In [ ]:
%time b_box = x.isel(latitude=slice(600,700), longitude=slice(650,750))

In [ ]:
%time b_box.isel(time=310).plot()

In [ ]:
%time plt.show()

## SILO data

Starting from some of code J Yu had authored:

In [ ]:
ds_desc = 'http://data-cbr.it.csiro.au/thredds/catalog/catch_all/Digiscape_Climate_Data_Portal/silo/climate/catalog.xml?dataset=allDatasetScan/Digiscape_Climate_Data_Portal/silo/climate/daily_rain.nc'
catalog = TDSCatalog(ds_desc)

In [ ]:
catalog.datasets

In [ ]:
dataset = catalog.datasets[0]

In [ ]:
type(dataset)

In [ ]:
blah

In [ ]:
ncss = dataset.subset()

In [ ]:
type(ncss)

In [ ]:
query = ncss.query()

In [ ]:
import pandas as pd
tt = pd.to_datetime("2014-01-01")

In [ ]:
query.lonlat_box(north=-40, south=-44, east=149, west=144).time(tt)

In [ ]:
query.accept('netcdf4')
query.variables('daily_rain')

In [ ]:
# With the following I may have bumped into https://github.com/Unidata/thredds-docker/issues/216 
data = ncss.get_data(query)

Try another approach:


In [ ]:
# Load the dataset
cat = catalog
dataset_name = sorted(cat.datasets.keys())[-1]

In [ ]:
dataset_name

In [ ]:
dataset = cat.datasets[dataset_name]
ds = dataset.remote_access(service='OPENDAP')

In [ ]:
from xarray.backends import NetCDF4DataStore

In [ ]:
ds = NetCDF4DataStore(ds)
ds = xr.open_dataset(ds)

In [ ]:
x = ds.daily_rain

In [ ]:
x

In [ ]:
x.isel(time=22000).plot()

In [ ]:
%time b_box = x.isel(lat=slice(600,700), lon=slice(650,750))

In [ ]:
%time b_box.isel(time=22000).plot()

In [ ]:
%time plt.show()

## Appendix

Below are attempts to use Numpy-ic ways to calculate quantile classes. No joy.

In [ ]:
convolve = np.vectorize(np.convolve, signature='(n),(m)->(k)')
convolve(np.eye(2), [1, 2])

In [ ]:
convolve = np.vectorize(np.convolve)
convolve(np.eye(2), [1, 2])

In [ ]:
convolve(np.eye(2), [1, 2])

In [ ]:
z = three_years_rains[-1]

In [ ]:
z.searchsorted(y)

In [ ]:
def searchsorted2d(a,b):
    m,n = a.shape
    max_num = np.maximum(a.max() - a.min(), b.max() - b.min()) + 1
    r = max_num*np.arange(a.shape[0])[:,None]
    p = np.searchsorted( (a+r).ravel(), (b+r).ravel() ).reshape(m,-1)
    return p - n*(np.arange(m)[:,None])

def searchsorted2d(mat,ensembles):
    m,n = mat.shape
    max_num = np.maximum(a.max() - a.min(), b.max() - b.min()) + 1
    r = max_num*np.arange(a.shape[0])[:,None]
    p = np.searchsorted( (a+r).ravel(), (b+r).ravel() ).reshape(m,-1)
    return p - n*(np.arange(m)[:,None])

In [ ]:
np.random.seed(123)
mat = np.sort(np.arange(12)).reshape(3, 4)

In [ ]:
mat = mat.reshape(3, 4)
mat

In [ ]:
mat[0]

In [ ]:
v_searchsorted = np.vectorize(np.searchsorted, signature='(n),(m)->(k)')

In [ ]:
v_searchsorted(mat, np.array([0.5, 5.5, 10.1]))

In [ ]:
v_searchsorted = np.vectorize(np.searchsorted, signature='(n,m),(n,m)->(n)')

In [ ]:
v_searchsorted(mat, np.array([0.5, 5.5]))

In [ ]:
np.searchsorted(np.array([-1, 0, 1, 2]), np.array([-1, 0, 1.1, 2]))

In [ ]:
y.groupby_bins(group, bins, right: bool = True, labels=None, precision: int = 3, include_lowest: bool = False, squeeze: bool = True, restore_coord_dims: Optional[bool] = None)

In [ ]:
z.shape, y.shape

In [ ]:
very_drier = z <= y[0,:,:]

In [ ]:
very_drier.plot()

In [ ]:
z[50,50], y[:,50, 50]

In [ ]:
np.argmin( (200 > y[:,50, 50].values) )

In [ ]:
3000 > y[:,50, 50].values

In [ ]:
np.argmin(z[50,50].values > y[:,50, 50].values)

In [ ]:
np.argmin(z[50,50].values < y[:,50, 50].values)

In [ ]:
(z[50,50].values > y[:,50, 50].values) , (z[50,50].values < y[:,50, 50].values)

In [ ]:
np.eye(4)

In [ ]:
np.argmax(z[50,50].values > y[:,50, 50].values)

In [ ]:
np.argmax(z[50,50].values < y[:,50, 50].values)

In [ ]:
np.searchsorted(y[:,50, 50].values, z[50,50].values)

In [ ]:
np.searchsorted(np.arange(10), 23)

In [ ]:
def searchsorted(a, b):
    func = lambda x, y: np.searchsorted(x[:,,].values, y[].values)
    return xr.apply_ufunc(func, a, b)

In [ ]:
xr.apply_ufunc

In [ ]:
dim = 'time'
xr.apply_ufunc(np.searchsorted,
                   y, z,
                   input_core_dims=[[dim], []])

In [ ]:
import scipy.stats

def earth_mover_distance(first_samples,
                         second_samples,
                         dim='ensemble'):
    return apply_ufunc(scipy.stats.wasserstein_distance,
                       first_samples, second_samples,
                       input_core_dims=[[dim], [dim]],
                       vectorize=True)